In [1]:
import pandas as pd 

DVF = pd.read_csv('dvf0.csv')

In [3]:
prix_m2_commune = DVF.groupby(['nom_commune', 'annee', 'type'])['prixm2'].mean().reset_index()
prix_m2_commune.to_csv("dvf.csv", index=False)

In [13]:
prix_m2_commune

,nom_commune,annee,type,prixm2
0,Aast,2019,Maison,95.000000
1,Aast,2020,Maison,1235.000000
2,Aast,2021,Maison,1699.000000
3,Aast,2022,Maison,2267.750000
4,Aast,2023,Maison,1328.333333
...,...,...,...,...
182959,Œuilly,2019,Maison,1343.500000
182960,Œuilly,2020,Maison,1700.750000
182961,Œuilly,2021,Maison,1826.625000
182962,Œuilly,2022,Maison,1739.400000


In [1]:
# importing pandas 
import pandas as pd 
  
# merging two csv files 
df = pd.concat( 
    map(pd.read_csv, ['dvf.csv', 'dvf1.csv','dvf2.csv', 'dvf3.csv','dvf4.csv']), ignore_index=True) 


In [3]:
df.to_csv('dvf0.csv', index=False)

In [5]:
import pandas as pd
import numpy as np

DVF = pd.read_csv('2019.csv')

: 

In [2]:
# Étape 1 : Filtrer uniquement les ventes de maisons et d'appartements
etape1 = DVF[DVF['nature_mutation'] == 'Vente']
etape1bis = etape1[(etape1['type_local'] == 'Maison') | (etape1['type_local'] == 'Appartement')]

# Étape 2 : Sélection et renommage des colonnes importantes
etape2 = etape1bis[['id_mutation', 'numero_disposition', 'id_parcelle', 'date_mutation', 'nature_mutation',
    'nom_commune', 'code_departement', 'type_local', 'surface_reelle_bati', 
    'nombre_pieces_principales', 'valeur_fonciere', 'latitude', 'longitude']]
etape2.columns = ['id', 'disposition', 'parcelle', 'date', 'nature', 'nom_commune', 'departement', 
    'type', 'surface', 'piece', 'prix', 'latitude', 'longitude']

# Étape 3 : Nettoyage des données
etape2.replace("", np.nan, inplace=True)
etape3 = etape2.dropna()

# Étape 4 : Supprimer les mutations multiventes (transactions complexes)
unique = etape3.drop_duplicates(subset=['id', 'prix', 'surface'])
nbunique = unique.groupby('id').size().reset_index(name='nb')
etape4 = nbunique[nbunique['nb'] == 1]

# Étape 5 : Jointure avec les informations complètes
merge = pd.merge(etape4, etape3, on="id")
etape5 = merge.drop_duplicates(subset="id").filter(
items=['id', 'date', 'type', 'nature', 'nom_commune', 'prix', 'surface', 'piece'])

# Conversion des types de données
etape5['prix'] = etape5['prix'].astype(int)
etape5['surface'] = etape5['surface'].astype(float)
etape5['piece'] = etape5['piece'].astype(float)

# Supprimer les valeurs aberrantes
etape6 = etape5[(etape5['prix'].between(15000, 5000000)) & 
((etape5['type'] == 'Appartement') & etape5['surface'].between(10, 250)) |
((etape5['type'] == 'Maison') & etape5['surface'].between(10, 500))]

# Création de la variable prix/m²
etape6['prixm2'] = (etape6['prix'] / etape6['surface']).round()

# Transformation de la date en année
etape6['annee'] = etape6['date'].astype(str).str[:4]

C:\Users\louis\AppData\Local\Temp\ipykernel_5376\2229743955.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  etape2.replace("", np.nan, inplace=True)
C:\Users\louis\AppData\Local\Temp\ipykernel_5376\2229743955.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  etape6['prixm2'] = (etape6['prix'] / etape6['surface']).round()
C:\Users\louis\AppData\Local\Temp\ipykernel_5376\2229743955.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [3]:
etape6['nom_commune'].value_counts()

nom_commune
Toulouse               9058
Nice                   8423
Nantes                 6216
Montpellier            5312
Bordeaux               4839
                       ... 
Carcanières               1
Prats-de-Sournia          1
Mantet                    1
Vaudherland               1
Épinay-Champlâtreux       1
Name: count, Length: 28795, dtype: int64

In [4]:
etape6.to_csv('dvf4.csv', index=False)